## Top 100 market cap - coingecko

In [ ]:
import requests
import pandas as pd
import json

In [ ]:


url = "https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd"
response = requests.get(url)

data = json.loads(response.content)
top100 = pd.DataFrame(data)

In [ ]:
top100.to_csv('top_100_cmc.csv')

In [ ]:
top100 = pd.read_csv('top_100_cmc.csv')

## ccxt - binance

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ccxt

In [ ]:
import ccxt
import pandas as pd
from datetime import datetime, timedelta

ex = ccxt.binance()
ex.options['maxRetriesOnFailureDelay'] = 1000

# Define the time range
start_date = '2017-08-17 00:00:00'
start_ts = ex.parse8601(start_date)
end_date = datetime.now().isoformat()
end_ts = ex.parse8601(end_date)


# Function to fetch OHLCV data in batches
def fetch_ohlcv_in_batches(exchange, symbol, timeframe, since, limit):
    all_ohlcv = []
    while since < end_ts:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since=since, limit=limit)
        if not ohlcv:
            break
        all_ohlcv.extend(ohlcv)
        since = ohlcv[-1][0] + 1  # Increment since to the last timestamp + 1

        
    return all_ohlcv

def ohlcv_to_df(ohlcv):
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    return df

# Fetch data
# ohlcv = ohlcv_to_df(fetch_ohlcv_in_batches(ex, 'BTC/USDT', '1d', start_ts, 1000))
# ohlcv.to_csv('btc.csv')

In [ ]:
ohlcvs = {}

for sym in list(top100['symbol']):
        print(f'fetching {sym}')
        try:
            ohlcv_df = ohlcv_to_df(fetch_ohlcv_in_batches(ex, f'{sym.upper()}/USDT', '1d', start_ts, 1000)) 
        except Exception as e:
            print(f"An unexpected error occurred: {sym} - {e}")

        ohlcvs[sym] = ohlcv_df
         

## Save & Import data

In [ ]:
import pickle

def save_dict_to_pickle(data_dict, filename):
    """
    Save a dictionary with DataFrame values to a pickle file.
    
    Parameters
    ----------
    data_dict : dict
        Dictionary with string keys and DataFrame values.
    filename : str
        The path to the file where the data will be saved.
    """
    with open(filename, 'wb') as f:
        pickle.dump(data_dict, f)

save_dict_to_pickle(ohlcvs, 'input.pkl')


In [ ]:
import pickle

def load_dict_from_pickle(filename):
    """
    Load a dictionary with DataFrame values from a pickle file.
    
    Parameters
    ----------
    filename : str
        The path to the file from which the data will be loaded.
    
    Returns
    -------
    data_dict : dict
        Dictionary with string keys and DataFrame values.
    """
    with open(filename, 'rb') as f:
        data_dict = pickle.load(f)
    return data_dict

# Example usage
ohlcvs = load_dict_from_pickle('input.pkl')
# print(ohlcvs)

### Remove irrelevant token

In [ ]:
for k in list(ohlcvs.keys()):
    if k in ['steth',
            'weeth',
            'ezeth',
            'reth',
            'meth',
            'eeth',
            'rseth']:
        del ohlcvs[k]

    if 'usd' in k:
        del ohlcvs[k]


## Visualize 

### MPF

In [ ]:
import mplfinance as mpf
mpf.plot(ohlcvs['sol'], type='line', volume=True, style='yahoo')
plt.show()

### Plotly

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=ohlcvs['eth'].index,
                open=ohlcvs['eth'].open, high=ohlcvs['eth'].high,
                low=ohlcvs['eth'].low, close=ohlcvs['eth'].close)])

fig.update_layout(title='OHLCV Data',
                   xaxis_title='Date',
                   yaxis_title='Price',
                   xaxis_rangeslider_visible=False)

fig.show()